## (1) Background

Consider the sum of a normal $Z_1 \sim N(\mu_1,\tau_1^2)$ and a truncated normal $Z_2 \sim TN(\mu_2,\tau_2^2,a,b)$, $W=Z_1+Z_2$. How can this normal-truncated sum (NTS) distribution be characterized? One approach for doing inference on $W$ is simply to sample from it, as sampling algorithms for both $Z_1$ and $Z_2$ are found in most statistical software. However, having an analytical or numerical method for calculating the density (PDF) and cumulative distribution (CDF) of $W$ has several benefits over the sampling approach. This includes including vectorization, reproducibility across software programs, and faster compute times (especially for the tails of the distribution). This post will provide `python` code to calculate the PDF and CDF of an arbitrary NTS. The value of distribution will be highlighted with three use cases: i) quality control, ii) two-stage hypothesis testing, and iii) data carving for selective inference. 

This post will make extensive use of the theoretical results from [Arnold et. al (1993)](https://link.springer.com/article/10.1007/BF02294652) (hereafter Arnold) and [Kim (2006)](https://www.kss.or.kr/jounalDown.php?IDX=831) (hereafter Kim). Sections (2) and (3) are not original work, but instead show how to translate these research papers into usable code. The two-stage hypothesis testing and data carving are unique in making the connection between these statistical problems and the NTS distribution. 

## (2) Integrating a bivariate normal CDF

The CDF of a bivarate normal (BVN) will turn out to be necessary to calculate the CDF of a NTS. While there is no closed-form solution to the CDF of a BVN (unless it's from the origin), the integration problem can be quickly solved by numerical methods. Additionally, if the user is willing to accept a non-exact solution, there are approximation methods which yield an analytical solution that can be easily vectorized. This section will briefly review both approaches. 

Following the notation of the literature, the orthant probability of a standard BVN[[1]] $L(h,k,\rho)=P(X_1\geq h, X_2\geq k)$ is equivalent to solving the following integral introduced by [Sheppard](https://royalsocietypublishing.org/doi/10.1098/rsta.1899.0003) 130 years ago:

$$
\begin{align}
L(h,k,\rho) &= \frac{1}{2\pi\sqrt{1-\rho^2}} \int_h^\infty \int_k^\infty \exp \Bigg[ - \frac{x^2-2\rho x y + y^2}{2(1-\rho^2)} \Bigg] dx dy  \\
&= \frac{1}{2\pi} \int_{\arccos(\rho)}^{\pi} \exp \Bigg[ - \frac{h^2+k^2-2hk\cos(\theta)}{2\sin^2(\theta)} \Bigg] d\theta \tag{1}
\end{align}
$$
\label{eq:sheppard}

Note that the orthant probability can easily be related to the CDF: $F(h,k,\rho)=P(X_1\leq h, X_2\leq k) = L(h,k,\rho) + \Phi(h) + \Phi(k) - 1$. The CDF and PDF of a standard normal are denoted by $\Phi(\cdot)$ and $\phi(\cdot)$, respectively. What is interesting about $\eqref{eq:sheppard}$ is that the problem of integrating out both $X_1$ and $X_2$ can been reduced to a single variable of integration. 

In `python`, the `scipy.integrate.quad` can be used to quickly solve $\eqref{eq:sheppard}$ quickly. As will be shown, the method turns out to produce results  identical to that of the `scipy.stats.multivariate_normal.cdf` function. Even though there is no closed-form solution, [Cox 1991](https://www.jstor.org/stable/1403446) showed that the estimate of $L(\cdot)$ could be approximated using a simple Taylor expansion.

$$
\begin{align}
L(h,k,\rho) &= P(X_1 \geq h, X_2 \geq k) = P(X_1 \geq h) P(X_2 \geq k | X_1 \geq h) \\
&= \Phi(-h) E\Bigg[ \Phi\Bigg(\frac{\rho X_1 - k}{\sqrt{1-\rho^2}} \Bigg) \Bigg| X_1 \geq h \Bigg] \\
&\approx \Phi(-h) \Phi\Bigg( \frac{\rho\phi(h)/\Phi(-h)-k}{\sqrt{1-\rho^2}} \Bigg) \tag{2}
\end{align} 
$$
\label{eq:cox}

Cox showed that this approximation method works well for reasonable ranges of $\rho$ (absolute coefficient less than 90%).  For users willing to trade off accuracy for speed, $\eqref{eq:cox}$ allows for rapid vectorization cross different values of $\rho$, $h$, or $k$. The code block below will define a `BVN` class that can be used to calculate the orthant probability and perform sampling. 

In [12]:
import numpy as np
import pandas as pd
from scipy.stats import norm, truncnorm
from scipy.stats import multivariate_normal as MVN
from scipy.linalg import cholesky
from scipy.integrate import quad
from scipy.optimize import minimize_scalar

class BVN():
    # mu=np.array([1,2]);sigma=np.array([2,3]); rho=0.5; # del mu, sigma, rho, seed
    def __init__(self, mu, sigma, rho):
        """
        mu: array of means
        sigma: array of variances
        rho: correlation coefficient
        """
        if isinstance(mu,list):
            mu, sigma = np.array(mu), np.array(sigma)
        assert mu.shape[0]==sigma.shape[0]==2
        assert np.abs(rho) <= 1
        self.mu = mu.reshape([1,2])
        self.sigma = sigma.flatten()
        od = rho*np.sqrt(sigma.prod())
        self.rho = rho
        self.Sigma = np.array([[sigma[0],od],[od, sigma[1]]])
        self.A = cholesky(self.Sigma) # A.T.dot(A) = Sigma

    # size=1000;seed=1234 # del size, seed
    def rvs(self, size, seed=None):
        """
        size: number of samples to simulate
        seed: to pass onto np.random.seed
        """
        np.random.seed(seed)
        X = np.random.randn(size,2)
        Z = self.A.T.dot(X.T).T + self.mu
        return Z

    def imills(self, a):
        return norm.pdf(a)/norm.cdf(-a)

    def sheppard(self, theta, h, k):
        return (1/(2*np.pi))*np.exp(-0.5*(h**2+k**2-2*h*k*np.cos(theta))/(np.sin(theta)**2))

    # self=dist_BVN; h, k = np.array([1,2,3]), np.array([2,3,4])
    def orthant(self, h, k, method='scipy'):
        # P(X1 >= h, X2 >=k)
        assert method in ['scipy','cox','sheppard']
        if isinstance(h,int) or isinstance(h, float):
            h, k = np.array([h]), np.array([k])
        else:
            assert isinstance(h,np.ndarray) and isinstance(k,np.ndarray)
        assert len(h) == len(k)
        # assert np.all(h >= 0) and np.all(k >= 0)
        # Calculate the number of standard deviations away it is        
        Y = (np.c_[h, k] - self.mu)/np.sqrt(self.sigma)
        Y1, Y2 = Y[:,0], Y[:,1]

        # (i) scipy: L(h, k)=1-(F1(h)+F2(k))+F12(h, k)
        if method == 'scipy':
            sp_bvn = MVN([0, 0],[[1,self.rho],[self.rho,1]])
            pval = 1+sp_bvn.cdf(Y)-(norm.cdf(Y1)+norm.cdf(Y2))
            return pval 
        
        # A Simple Approximation for Bivariate and Trivariate Normal Integrals
        if method == 'cox':
            mu_a = self.imills(Y1)
            root = np.sqrt(1-self.rho**2)
            xi = (self.rho * mu_a - Y2) / root
            pval = norm.cdf(-Y1) * norm.cdf(xi)
            return pval

        if method == 'sheppard':
            pval = np.array([quad(self.sheppard, np.arccos(self.rho), np.pi, args=(y1,y2))[0] for y1, y2 in zip(Y1,Y2)])
            return pval

mu = np.array([1,2])
sigma = np.array([0.5,2])
dist_BVN = BVN(mu,sigma,rho=0.4)
h, k = 2, 3
pval_scipy = dist_BVN.orthant(h, k, method='scipy')[0]
pval_sheppard = dist_BVN.orthant(h, k, method='sheppard')[0]
pval_cox = dist_BVN.orthant(h, k, method='cox')[0]
nsim = 1000000
pval_rvs = dist_BVN.rvs(nsim)
pval_rvs = pval_rvs[(pval_rvs[:,0]>h) & (pval_rvs[:,1]>k)].shape[0]/nsim
methods = ['scipy','sheppard','cox','rvs']
pvals = [pval_scipy, pval_sheppard, pval_cox, pval_rvs]
pd.DataFrame({'method':methods,'orthant':pvals})

,method,orthant
0,scipy,0.040615
1,sheppard,0.040615
2,cox,0.040670
3,rvs,0.040819


Notice that the output results are identical using either the `multivariate_normal` class or directly calling integration solvers.  Figure 1A below shows that the Cox-method is very close to the numerical solution, although there are no definitive error bounds (in Cox's paper he found that the worst-case empirical error was around 10%). Although the figure is not shown, there is no improvement in the run time using the `sheppard` method over `scipy`'s built in MVN distribution for a single estimate (and `scipy` does better for a vector/matrix of value. However, there are substantial gains in the run time for vectorization across a matrix of $h,k$ values using Cox's methods (Figure 1B).

<center><h4>Figure 1: BVN comparison </h4></center>
<table>
    <tr>
        <td> <center>A: Orthant probabilities   </center> </td>
        <td> <center>B: Vectorized run-times </center> </td>
    <tr>
        <td> <img src="figures/gg_pval.png" style="width: 80%"/> </td>
        <td> <img src="figures/gg_rvec.png" style="width: 90%"/> </td>
    </tr>
</table>

While this section might have seemed like something of a detour, its usefulness will made apparent soon. 

## (3) Deriving $f_W$ and $f_W$

The first step to characterize the NTS distribution is to understand the distribution of a truncated bivariate normal distrubion (TBVN). 

$$
\begin{align*}
(X_1, X_2) &\sim \text{TBVN}(\theta, \Sigma, \rho, a, b) \\
E([X_1,X_2]) &= [\theta_1, \theta_2] \\
V([X_1,X_2]) &= \begin{pmatrix} \sigma_1^2 & \rho \sigma_1\sigma_2 \\ \rho \sigma_1\sigma_2 & \sigma_2^2  \end{pmatrix} \\
X_1 &\in \mathbb{R}, \hspace{3mm} X_2 \in [a,b]
\end{align*}
$$

Notice that the TBVN takes the same formulation as a bivariate normal (a mean vector, a covariance matrix, and a correlation coefficient $\rho$) except that the random variable $X_2$ term is bound between $a$ and $b$. Arnold showed that the marginal density of the non-truncated random variable $X_1$ could be written as follows:

$$
\begin{align}
f_W(x) &= f_{X_1}(x) = \frac{\phi(m_1(x)) \Bigg[ \Phi\Big(\frac{\beta-\rho \cdot m_1(x)}{\sqrt{1-\rho^2}}\Big) - \Phi\Big(\frac{\alpha-\rho \cdot m_1(x)}{\sqrt{1-\rho^2}}\Big) \Bigg] }{\sigma_1\cdot Z} \tag{3} \\ 
m_1(x) &= (x - \theta_1)/\sigma_1  \\
\alpha &= \frac{a-\theta_2}{\sigma_2}, \hspace{3mm} \beta = \frac{b-\theta_2}{\sigma_2}  \\
Z &= \Phi(\beta) - \Phi(\alpha)
\end{align}
$$
\label{eq:arnold}

Kim showed that a NTS could be written as a TBVN by a simple change of variables:

$$
\begin{align*}
X_1 &= Z_1 + Z_2^u \\ 
X_2 &= Z_2^u, \hspace{3mm} X_2 \in [a,b] \\
Z_2^u &\sim N(\mu_2,\tau_2^2) \\
\theta &= [\mu_1 + \mu_2, \mu_2] \\
\Sigma &= \begin{pmatrix} \tau_1^2 + \tau_2^2 & \rho \sigma_1\sigma_2 \\ \rho \sigma_1\sigma_2 & \tau_2^2  \end{pmatrix} \\
Z_1 + Z_2 &= W \sim NTS(\theta(\mu),\Sigma(\tau^2), a, b)
\end{align*}
$$

Where $Z_2^u$ is the non-truncated version of $Z_2$. Clearly this specification of the TBVN is equivalent to the NTS and the marginal distribution of $X_1$ is equivalent to the PDF of $W$. Kim showed that the integral of $\eqref{eq:arnold}$ was equivalent to solving the orthant probabilities of a standard BVN : 

$$
\begin{align}
F_W &= F_{X_1}(x) = 1 - \frac{L(m_1(x),\alpha,\rho) - L(m_1(x),\beta,\rho)}{Z} \tag{4}
\end{align}
$$
\label{eq:kim}

Hence, the PDF for a NTS can be solved analytically, and CDF can be solved using either numerical methods for the exact univariate integral in $\eqref{eq:sheppard}$ or Cox's approximation method in $\eqref{eq:cox}$. The code block below provides the python code necessary to calculate $\eqref{eq:arnold}$ and $\eqref{eq:kim}$. Although I have not implemented an efficient solution, the `NTS` class provides a quantile function $F^{-1}_W(p)=\inf_w F_W(w) \geq p$ that relies on [Golden's method](https://en.wikipedia.org/wiki/Golden-section_search) for univariate optimization by querying the `cdf` attribute until convergence. 

In [13]:
class NTS():
    def __init__(self, mu, tau, a, b):
        """
        mu: array of means
        tau: array of standard errors
        rho: correlation coefficient
        """
        assert mu.shape[0]==tau.shape[0]==2
        # Assign parameters
        self.mu, self.tau = mu.flatten(), tau.flatten()
        self.a, self.b = a, b
        # Truncated normal (Z2)
        self.alpha = (self.a - self.mu[1]) / self.tau[1]
        self.beta = (self.b - self.mu[1]) / self.tau[1]
        self.Z = norm.cdf(self.beta) - norm.cdf(self.alpha)
        self.Q = norm.pdf(self.alpha) - norm.pdf(self.beta)
        # Average will be unweighted combination of the two distributions
        self.mu_W = self.mu[0] + self.mu[1] + self.tau[1]*self.Q/self.Z
        # Distributions
        self.dist_X1 = norm(loc=self.mu[0], scale=self.tau[0])
        self.dist_X2 = truncnorm(a=self.alpha, b=self.beta, loc=self.mu[0], scale=self.tau[1])
        # W
        self.theta1 = self.mu.sum()
        self.theta2 = self.mu[1]
        self.sigma1 = np.sqrt(np.sum(self.tau**2))
        self.sigma2 = self.tau[1]
        self.rho = self.sigma2/self.sigma1

    def pdf(self, x):
        term1 = self.sigma1 * self.Z
        m1 = (x - self.theta1) / self.sigma1
        term2 = (self.beta-self.rho*m1)/np.sqrt(1-self.rho**2)
        term3 = (self.alpha-self.rho*m1)/np.sqrt(1-self.rho**2)
        f = norm.pdf(m1)*(norm.cdf(term2) - norm.cdf(term3)) / term1
        return f

    def cdf(self, x, method='scipy'):
        if isinstance(x, list):
            x = np.array(x)
        if isinstance(x, float) or isinstance(x, int):
            x = np.array([x])
        nx = len(x)
        m1 = (x - self.theta1) / self.sigma1
        bvn = BVN(mu=[0,0],sigma=[1,1],rho=self.rho)
        orthant1 = bvn.orthant(m1,np.repeat(self.alpha,nx),method=method)
        orthant2 = bvn.orthant(m1,np.repeat(self.beta,nx),method=method)
        return 1 - (orthant1 - orthant2)/self.Z

    def ppf(self, p):
        if isinstance(p, list):
            p = np.array(p)
        if isinstance(p, float) or isinstance(p, int):
            p = np.array([p])
        # Set up reasonable lower bounds
        lb = self.mu_W - self.sigma1*4
        ub = self.mu_W + self.sigma1*4
        w = np.repeat(np.NaN, len(p))
        for i, px in enumerate(p):
            tmp = float(minimize_scalar(fun=lambda w: (self.cdf(w)-px)**2,method='bounded',bounds=(lb,ub)).x)
            w[i] = tmp
        return w

    def rvs(self, n, seed=1234):
        r1 = self.dist_X1.rvs(size=n,random_state=seed)
        r2 = self.dist_X2.rvs(size=n,random_state=seed)
        return r1 + r2

Next, let's generate NTS data with the following parameters: $\mu_1=1$, $\tau_1=1$ (for $Z_1$), $\mu_2=1$, $\tau_2=2$, $a=-1$, and $b=4$ (for $Z_2$).

In [14]:
# Demonstrated with example
mu1, tau1 = 1, 1
mu2, tau2, a, b = 1, 2, -1, 4
mu, tau = np.array([mu1, mu2]), np.array([tau1,tau2])**2
dist_NTS = NTS(mu=mu,tau=tau, a=a, b=b)
n_iter = 100000
W_sim = dist_NTS.rvs(n=n_iter,seed=1)
mu_sim, mu_theory = W_sim.mean(),dist_NTS.mu_W
xx = np.linspace(-5*mu.sum(),5*mu.sum(),n_iter)
mu_quad = np.sum(xx*dist_NTS.pdf(xx)*(xx[1]-xx[0]))
methods = ['Empirical','Theory', 'Quadrature']
mus = [mu_sim, mu_theory, mu_quad]
# Compare
print(pd.DataFrame({'method':methods,'mu':mus}))

       method        mu
0   Empirical  2.439796
1      Theory  2.438245
2  Quadrature  2.438245


The table above compares the empirical mean of the simulated data with the sum of the two location parameters of $Z_1$ and $Z_2$, as well as what we would estimate using a quadrature procedure with equation $\eqref{eq:arnold}$, as $E(W)=\int w F_W(w) dw$. Next, we can compare the actual and theoretical percentiles and quantiles against the empirical ones seen from `W_sim`. As expected, they are visually indistinguishable from each other.[[^2]]

<center><h4>Figure 2: NTS P-P & Q-Q plot </h4></center>
<img src="figures/gg_ppqq.png" style="width: 70%"/>

## (4.A) Application: Quality control

## (4.B) Two-stage testing

In a [previous post](http://www.erikdrysdale.com/regression_trial/) I discussed a two-stage testing strategy designed to validate a machine learning regression model. The framework can be generalized as follows: 1) estimate an upper-bound on the mean of a Gaussian, 2) use this upper bound as the null hypothesis on a new sample of data.[[^3]] Several simplifying assumptions need to be made including that the data are iid and from the same normal distribution: $S, T \overset{iid}{\sim} N(\delta,\sigma^2)$, and that $n$ and $m$ are large enough so that the difference between the normal and student-t distributions are sufficiently small. The statistical pipeline is as follows.

1. Estimate the mean and variance on the first Gaussian sample: $(S_{1},\dots,S_{n}) \sim N(\hat{\delta}_S,\hat{\sigma}^2/n)$
2. Estimate the $1-\gamma$ quantile of this distribution: $\hat{\delta}_0=\hat{\delta}_S+n^{-1/2}\cdot\hat{\sigma}\cdot\Phi^{-1}_{1-\gamma}$ to set the null hypothesis:
$$
\begin{align*}
H_0&: \delta \geq \hat{\delta}_0 \\
H_A&: \delta < \hat{\delta}_0 \\
\end{align*}
$$
3. Estimate mean and variance on second Gaussian sample: $(T_{1},\dots,T_{m}) \sim N(\hat{\delta}_T,\hat{\sigma}_m^2)$
4. Calculate a one-sided test statistic: $\hat{s} = \sqrt{m} \cdot (\hat{\delta}_T - \hat{\delta}_0) / \hat{\sigma}$
$$
\begin{align*}
\text{Reject }H_0&: \hat{s} < t_\alpha
\end{align*}
$$


Clearly $E(s)<0$ if $\tau >0.5$. The statistical information provided by this procedure is two-fold. First, $\delta_0$ has the property that it will be larger than the true mean $(1-\tau)$% of the time (this is only in the frequentist sense, so it says nothing about any realization $\hat{\delta}_0$ itself). Second, when the null is false (i.e. the bound holds) we will be able to compute the power in advance.  Now, where does the NTS fit into this? In order to bound the type-I error rate to $\alpha$, we need to know the distribution of $s$ when the null is true: $\delta > \hat{\delta}_0$:

$$
\begin{align*}
\hat{s} &= \frac{\hat{\delta}_T - \delta}{\hat{\sigma}_m} - \frac{\hat{\delta}_0 - \delta}{\hat{\sigma}_m} \hspace{2mm} \Big| \hspace{2mm} \hat{\delta}_0 > \delta  \\
&= N(0,1) - \frac{\hat{\delta}_S-\delta + \hat{\sigma}_n\cdot\Phi^{-1}_{1-\gamma}}{\hat{\sigma}_m} \hspace{2mm} \Big| \hspace{2mm} \hat{\delta}_S + \hat{\sigma}_n\cdot\Phi^{-1}_{1-\gamma} - \delta > 0 \\
&= N(0,1) + TN\big( - \sqrt{m/n} \cdot \Phi^{-1}_{1-\gamma}, m/n, 0, \infty  \big)
\end{align*}
$$

Hence the critical value $t_\alpha = F^{-1}_W(\alpha)$ can be found by inverting $\eqref{eq:kim}$ (i.e. the quantile function) with the following parameters from our original notation:

$$
\begin{align*}
\mu &= \big[ 0, - \sqrt{m/n} \cdot \Phi^{-1}_{1-\gamma}\big] \\
\tau^2 &= \big[1, m/n \big] \\
\hat{s} | H_0 &\sim NTS(\theta(\mu), \Sigma(\tau^2), 0, \infty ) \\ 
\hat{s} | H_A &\sim NTS(\theta(\mu), \Sigma(\tau^2), -\infty, 0 )
\end{align*}
$$

Notice that the distribution of the test statistic only depends on $m$, $n$, and $\gamma$. To the extent that the sample size is fixed, then the researcher can only control $\gamma$, with a higher value increasing the power of a test, but lowering the statistical information that it provides (because the upper bound is larger). The simulations below will draw $S, T \sim N(2,4)$, with $n=250$, ad $m=500$. We'll look at whether the `NTS` class we've created accurately captures the distribution of $\hat{s}$ when the null is either true or false.

In [20]:
delta, sigma2 = 2, 4
n, m = 250, 500
tau, alpha = 0.1, 0.05
mu_2stage = np.array([0, -np.sqrt(m/n)*norm.ppf(1-tau)])
tau_2stage = np.sqrt([1, m/n])
dist_2s_H0 = NTS(mu=mu_2stage,tau=tau_2stage, a=0, b=np.infty)
dist_2s_HA = NTS(mu=mu_2stage,tau=tau_2stage, a=-np.infty, b=0)
crit_val = dist_2s_H0.ppf(alpha)[0]

# Compare to simulation
nsim = 50000
np.random.seed(nsim)
S = delta+np.sqrt(sigma2)*np.random.randn(nsim,n)
T = delta+np.sqrt(sigma2)*np.random.randn(nsim,m)
delta1, delta2 = S.mean(1), T.mean(1)
sigS, sigT = S.std(1,ddof=1), T.std(1,ddof=1)
del S, T
delta0 = delta1 + (sigS/np.sqrt(n))*norm.ppf(1-tau)
shat = (delta2 - delta0)/(sigT/np.sqrt(m))
p_seq = np.arange(0.01,1,0.01)
s_null_H0 = shat[delta > delta0]
s_null_HA = shat[delta < delta0]
qq_emp_H0 = np.quantile(s_null_H0,p_seq)
qq_emp_HA = np.quantile(s_null_HA,p_seq)
qq_theory_H0 = dist_2s_H0.ppf(p_seq)
qq_theory_HA = dist_2s_H0.ppf(p_seq)
tmp1 = pd.DataFrame({'pp':p_seq,'emp':qq_emp_H0,'theory':qq_emp_H0,'Null':'H0'})
tmp2 = pd.DataFrame({'pp':p_seq,'emp':qq_emp_HA,'theory':qq_emp_HA,'Null':'HA'})
dat_2stage = pd.concat([tmp1, tmp2]).melt(['pp','Null'],None,'tt','qq')

<center><h4>Figure 3: Two-stage test statistic </h4></center>
<img src="figures/gg_2stage_alpha.png" style="width: 70%"/>

Figure 3 shows that the empirical and theoretical quantiles line up exactly as expected, showing that our NTS distribution is working with $\mu(m,n,\gamma)$ and $\tau(m,n)$. We can also explore how the power changes when $m$, $n$, and $\gamma$ are varied. In the results below, $m+n=750$.

## (4.C) Application: Data carving

<br> 
* * *

### Footnotes

[^1]: A standard BVN means that the means are zero, and covariance matrix has a value of one on the diagonals and $\rho$ on the off-diagonals.

[^2]: In reality, there are slight differences, they just cannot be seen on the figure.

[^3]: A lower bound can be studied just as easily as an upper bond.